# Fine-tuning GPT-2
## Write a conclusion given the title

## Train

In [1]:
# import libraries

!pip install transformers
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
from transformers import pipeline
import csv
import re
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.7 MB/s eta 0:00:00


In [2]:
# instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2',sep_token = '<|sep|>',truncation=True, max_length=600000)
tokenizer.model_max_length

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1024

In [4]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

# data loading (data already preprocessed by adding special tokens)
train_dataset,test_dataset,data_collator = load_dataset('train_gpt2_conclusion.txt','val_gpt2_conclusion.txt', tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (1647075 > 1024). Running this sequence through the model will result in indexing errors


In [5]:
# istantiate the model (gpt-2 small with 12-layer, 768-hidden, 12-heads, 117M parameters)
model = AutoModelWithLMHead.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1248: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Embedding(50258, 768)

In [6]:
training_args = TrainingArguments(
    output_dir="./model_parameters",  # define the output directory
    overwrite_output_dir=True,        # True if we want to overwrite the content of the output directory
    num_train_epochs=6,               # number of training epochs
    per_device_train_batch_size=16,   # batch size for training
    per_device_eval_batch_size=16,    # batch size for evaluation
    logging_strategy="steps",
    logging_steps=400,
    eval_steps=200,                   # number of update steps between two evaluations
    save_steps=600,                   # number of update steps before saving the update model 
    warmup_steps=200,                 # number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [7]:
import torch

# move the computation on the gpu, if available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cuda


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12867
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4830
  Number of trainable parameters = 124440576


Step,Training Loss
400,9.344600
800,3.702500
1200,3.556000
1600,3.509600
2000,3.405500
2400,3.396400
2800,3.314500


Saving model checkpoint to ./model_parameters/checkpoint-600
Configuration saved in ./model_parameters/checkpoint-600/config.json
Configuration saved in ./model_parameters/checkpoint-600/generation_config.json
Model weights saved in ./model_parameters/checkpoint-600/pytorch_model.bin
Saving model checkpoint to ./model_parameters/checkpoint-1200
Configuration saved in ./model_parameters/checkpoint-1200/config.json
Configuration saved in ./model_parameters/checkpoint-1200/generation_config.json
Model weights saved in ./model_parameters/checkpoint-1200/pytorch_model.bin
Saving model checkpoint to ./model_parameters/checkpoint-1800
Configuration saved in ./model_parameters/checkpoint-1800/config.json
Configuration saved in ./model_parameters/checkpoint-1800/generation_config.json
Model weights saved in ./model_parameters/checkpoint-1800/pytorch_model.bin
Saving model checkpoint to ./model_parameters/checkpoint-2400
Configuration saved in ./model_parameters/checkpoint-2400/config.json
Confi

In [ ]:
# Save
model_path = "./model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

## Generate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load
model_path = "./model"
model = AutoModelWithLMHead.from_pretrained(model_path)

In [ ]:
# Generate ans store new data
import pandas as pd
generator = pipeline('text-generation', model=model_path, tokenizer='gpt2', device=0)

def generate_conclusion(text):
    # generate the content
    t = generator(text, do_sample=True, max_length=512, top_k=50, top_p=0.50, num_return_sequences=1)[0]['generated_text']
    # remove the input string 
    t = t[len(text):]
    # search the first upper case letter and delete what was generate before it
    m = re.search("[A-Z]", t)
    if m is not None:
      m = m.start()
      t = t[m:]
    # remove \n and add . if necessary
    t = t.replace('\n', ' ').rsplit('. ', 1)[0] + '.'
    # remove additional parts generated
    if t.find('<|sep|>'):
      x = t.find('<|sep|>')
      t = t[:x].rsplit('. ', 1)[0] + '.'
    return t

titles = pd.read_csv('./gpt2.csv')['title']

# create the output file
with open("./gpt2_from_title_to_conclusion.csv",'w') as csvfile:
  # write the header
  writer = csv.DictWriter(csvfile, fieldnames=['title', 'conclusion'])
  writer.writeheader()
  # generate the content for each title and append it to the output file
  for elem in titles:
    writer.writerow({"title": elem,'conclusion': generate_conclusion(str(elem + '<|sep|>'))})